# Introduction

In this notebook, we'll learn how EMERGENT models parameter spaces and finds extrema. First, we'll study how EMERGENT chooses experimental data points to acquire, governed by the sampling method chosen by you, the user. Let's look at the base Sampling class:

In [ ]:
from emergent.samplers.sampling import Sampling
Sampling??

In order to define a sampling method, you should override the \_run() method, as we will look at shortly. For now, understand that this method simply executes a bunch of experiments and returns the points (parameter space coordinates) and costs (experimental results). After the data is acquired, the \_finish() method allows a model to be fit to the data, then goes to either the first, last, or best point sampled. The Sampling class also implements a standardized plot() method which returns a handle to a 2D plot of the parameter space (currently only 2D spaces are supported).

Now let's look at how to define a sampling method. Consider the Grid sampling class:

In [ ]:
from emergent.samplers import Grid
Grid??

As we can see, defining new sampling methods is easy - just override the \_run() method, then define any parameters you don't want to hard-code; these parameters will automatically appear in the GUI to be set before starting a run. In this case, the \_run() method creates a uniform grid between 0 and 1 in _N_ dimensions, runs the experiment at each point, and returns the results.

<div class="alert alert-block alert-warning">
Don't worry about the actual ranges - the sampling takes places in a normalized subspace from 0 to 1 along all coordinates, and EMERGENT automatically scales to the ranges defined by the Hub.range dictionary, also displayed in the GUI's network tree.
</div>


Grid sampling can be useful for exploring experiments with 1 or 2 knobs, but the number of iterations required grows exponentially in the number of dimensions. For experiments with many free parameters, we can sample more intelligently with the _online_ sampling method:

In [ ]:
from emergent.samplers import Online
Online??

The basic idea is that we attach a model to the sampling process - this could be based on physical intuition or just a general modeler like a Gaussian process. The sampling proceeds as follows:

1. Pre-sample some number of randomly generated points.
2. Fit the model to all acquired data.
3. Numerically minimize the _effective cost_ over the modeled surface to choose the next point.
4. Repeat step 3 for each point in a batch, then return to step 2, and repeat.

The _effective cost_ is the magic bullet that makes this work so well. Instead of sampling uniformly, we sample intelligently, using all of the knowledge gathered about the experiment to suggest the next point. But what function should we use for the effective cost? A good first guess is the amplitude of the parameter surface itself, i.e. seeking extrema where the signal-to-noise is high. However, if your model can also generate an uncertainty estimate for a point, then it works well to optimize some linear combination of the amplitude and the uncertainty. By iterating through various combinations, we can go from optimizer mode (seeking low amplitude) and explorer mode (seeking high uncertainty), refining the model very quickly.

Let's now examine how models are implemented in EMERGENT, starting with the base Model class:

In [ ]:
from emergent.models.model import Model
Model??

When defining a model, you should override the fit() and predict() methods. The former should fit the model based on all acquired data so far, while the latter should generate a prediction of the experimental result (and possibly the uncertainty) at a given state dictionary. The Model class implements a number of other functions too, including generation of the next_sample() in terms of the explorer/optimizer tradeoff _b_ and the ability to plot() the modeled parameter surface in 2D.

Now let's look at a specific implementation of a model, Gaussian process regression:

In [ ]:
from emergent.models import GaussianProcess
GaussianProcess??

Just as with defining a Sampling subclass, it is very simple to define a new model. In this case, we initialize a regressor in the __init__ method, then override the Model.fit() and Model.predict() methods with the specific syntax required for the GaussianProcessRegressor class imported from scikit-learn.

# Running an optimization
Now let's try actually running an optimization! First, initialize a demo network by running the Getting Started notebook:

In [ ]:
%run "./Getting started.ipynb"
import requests
base_url = 'http://localhost:6000'
while True:
    try:
        r=requests.get(base_url)
        if r.text == 'EMERGENT API':
            break
    except:
        continue
print(r.text)

In order to start a process, we POST a settings dictionary to the /run endpoint containing all of the info necessary to define the process. All of the following is handled much simpler by the GUI, but this tutorial demonstrates the exact steps required to manually launch an optimization through the API. Let's construct a command for a grid search optimization of the "gaussian" experiment attached to our hub:

In [ ]:
hub = network.hubs['hub']
hub.range['thing'] = {'X': {'min': -2, 'max': 2}, 'Y': {'min': -2, 'max': 2}}
thing = hub.children['thing']
state = {'thing': thing.state}
settings = {'hub': hub.name, 'state': state}

settings['experiment'] = {'name': 'gaussian',
                          'params': {'sigma_x': 0.3, 
                                     'sigma_y': 0.8, 
                                     'x0': 0.3, 
                                     'y0': 0.6, 
                                     'noise':0} }
settings['sampler'] = {'name': 'Grid',
                       'params': {'Steps': 20, 'Sweeps': 1} }

settings['process'] = {'type': 'model', 'end at': 'Best point', 'callback': None}

Now let's submit this process via the API:

In [ ]:
import requests
requests.post('http://127.0.0.1:6000/run', json=settings)

Because we specified the Grid sampler with 20 steps, the optimization was a crude brute force search on a 20x20 grid, converging to near the peak after 400 experimental cycles. 

## Online optimization
We could solve this problem more efficiently with online Gaussian process optimization:

In [ ]:
hub = network.hubs['hub']
hub.range['thing'] = {'X': {'min': -2, 'max': 2}, 'Y': {'min': -2, 'max': 2}}
thing = hub.children['thing']
state = {'thing': thing.state}
settings = {'hub': hub.name, 
            'state': state}
settings['experiment'] = {'name': 'gaussian',
                          'params': {'sigma_x': 0.3, 
                                     'sigma_y': 0.8, 
                                     'x0': 0.3, 
                                     'y0': 0.6, 
                                     'noise':0.0} }
settings['sampler'] = {'name': 'Online',
                       'params': {'Presampled points': 5, 
                                  'Iterations': 10,
                                  'Batch size': 5,
                                  'Tolerance': 0,
                                  'Mode': 'Hybrid'} }
settings['model'] = {'name': 'GaussianProcess',
                     'params': {'Amplitude': 1,
                                'Length scale': 1,
                                'Noise': 0,
                                'Leash': 1} }

settings['process'] = {'type': 'model', 'end at': 'Best point', 'callback': None}

In [ ]:
network.actuate({'hub': {'thing': {'X':0, 'Y':0}}})
import requests
requests.post('http://127.0.0.1:6000/run', json=settings)

In [ ]:
samplers = requests.get('http://127.0.0.1:6000/hubs/hub/samplers').json()
requests.get('http://127.0.0.1:6000/hubs/hub/samplers/%s/plot/history'%samplers[len(samplers)-1])

Now the model converged within 1% tolerance much quicker! However, we can do better than this: while Gaussian processes can model virtually any complex experimental parameter space, the training requires fitting many degrees of freedom. If we can restrict the degrees of freedom by suggesting a model that's close to the experimental landscape, we can vastly improve the training time:

In [ ]:
hub = network.hubs['hub']
thing = hub.children['thing']
state = {'thing': thing.state}
settings = {'hub': hub.name, 
            'state': state}
settings['experiment'] = {'name': 'gaussian',
                          'params': {'sigma_x': 0.3, 
                                     'sigma_y': 0.8, 
                                     'x0': 0.3, 
                                     'y0': 0.6, 
                                     'noise':0} }
settings['sampler'] = {'name': 'Online',
                       'params': {'Presampled points': 20, 
                                  'Iterations': 5,
                                  'Batch size': 5,
                                  'Tolerance': 0.01} }
settings['model'] = {'name': 'Nonlinear',
                     'params': {'Leash': 0.25} }

settings['process'] = {'type': 'model', 'end at': 'Best point', 'callback': None}

In [ ]:
network.actuate({'hub': {'thing': {'X':0, 'Y':0}}})
import requests
requests.post('http://127.0.0.1:6000/run', json=settings)

Our targeted Gaussian model converged with fewer iterations and less computation time per training cycle than the more general Gaussian process model!

# Visualizing results
The results of the last two experiments can be accessed through the API. First, we find the process ID corresponding to the run we want to look at; these are listed in chronological order at the following endpoint:

In [ ]:
import requests
samplers = requests.get('http://127.0.0.1:6000/hubs/hub/samplers').json()

Now let's plot the data acquired by the first sampler:

In [ ]:
requests.get('http://127.0.0.1:6000/hubs/hub/samplers/%s/plot/data'%samplers[0])

Since we specified a model to fit in the second process, we can plot not only the raw data but also the modeled surface through the following endpoint:

In [ ]:
requests.get('http://127.0.0.1:6000/hubs/hub/samplers/%s/plot/model'%samplers[1])

Notice that the parameter space is resolved with decent accuracy and high resolution in the second case, despite the fact that we didn't acquire as much experimental data! If you know the functional form of your experiment's parameter space, you should write a Model and use it to generate points with the Online sampler; if not, you can always use a black-box modeler like GaussianProcess.

# Exporting and importing models
Training models with many degrees of freedom can require a lot of data, especially when the dimensionality of the parameter space is large. Luckily, EMERGENT offers the ability to import weights from previously-trained models as initial guesses to speed up convergence. After running an optimization, we can export the weights using the Model.\_export() abstract method:

In [ ]:
hub.samplers[1].model._export()

Each Model class has a unique file extension labeling its saved weights: for example, the .gpr extension is associated with the GaussianProcess model. We can get all trained weights in the network's data folder with the following endpoint:

In [ ]:
import requests
requests.get('http://localhost:6000/models/GaussianProcess/weights').text

We can load the model we exported above by specifying it in the settings dictionary. 

In [ ]:
hub = network.hubs['hub']
hub.range['thing'] = {'X': {'min': -2, 'max': 2}, 'Y': {'min': -2, 'max': 2}}
thing = hub.children['thing']
state = {'thing': thing.state}
settings = {'hub': hub.name, 
            'state': state}
settings['experiment'] = {'name': 'gaussian',
                          'params': {'sigma_x': 0.3, 
                                     'sigma_y': 0.8, 
                                     'x0': 0.3, 
                                     'y0': 0.8,
                                     'noise':0.0} }
settings['sampler'] = {'name': 'Online',
                       'params': {'Presampled points': 20, 
                                  'Iterations': 5,
                                  'Batch size': 5,
                                  'Tolerance': 0,
                                  'Mode': 'Hybrid'} }
settings['model'] = {'name': 'GaussianProcess',
                     'params': {'Amplitude': 1,
                                'Length scale': 1,
                                'Noise': 0,
                                'Weights': 'model.gp',
                                'Leash': 1} }

settings['process'] = {'type': 'model', 'end at': 'Best point', 'callback': None}


In [ ]:
network.actuate({'hub': {'thing': {'X':0, 'Y':0}}})
import requests
requests.post('http://127.0.0.1:6000/run', json=settings)

In [ ]:
samplers = requests.get('http://127.0.0.1:6000/hubs/hub/samplers').json()
requests.get('http://127.0.0.1:6000/hubs/hub/samplers/%s/plot/history'%samplers[len(samplers)-1])

In [ ]:
network.hubs['hub'].samplers[1].id